In [4]:
import tensorflow as tf
import numpy as np

# weight를 session에 넣지 않고 보기위해, 2.0에는 필수로.
# tf.enable_eager_execution()

# 1. Normal Softmax

In [5]:
def sotfmax_regression_1():
    # 1행은 공부시간, 2행은 출석
    x = [[1., 2.],   # C
         [2., 1.],
         [3., 5.],   # B
         [5., 3.],
         [7., 9.],   # A
         [9., 7.]]
    # [1,0,0] A학점
    # [0,1,0] B학점
    y = [[0,0,1],
         [0,0,1],
         [0,1,0],
         [0,1,0],
         [1,0,0],
         [1,0,0]]

    w = tf.Variable(tf.random_uniform([2,3]))
    # bias는 class의 갯수만큼 정한다.
    b = tf.Variable(tf.random_uniform([3]))
    print(w)

    # (6, 3) = (6, 2) @  (2, 3)
    z = tf.matmul(x, w) + b
    hx = tf.nn.softmax(z)

    # tensorflow 2.0d에서 정리 될것,  logit=예측  label을 결과괌
    # logit에 z를 하는것은 hx를 전달하지 않아도 됨, 자체적으로 함
    loss_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits=z, labels=y)
    loss   = tf.reduce_mean(loss_i)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    train = optimizer.minimize(loss= loss)

    # sess로 변수의 값을 알수 있다.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for i in range(100):
        sess.run(train)
        print(i, sess.run(loss))

    preds = sess.run(hx)
    print(preds)
    print(y)

    preds_arg = np.argmax(preds, axis=1) # 1: 수평, 0: 수직
    y_arg = np.argmax(y, axis=1)
    print(preds_arg)
    print(y_arg)

    equals = (preds_arg == y_arg)
    print(equals)
    print('acc: ', np.mean(equals))

    # 파이썬의 list는 broadcasting기능이 없어서, numpy array로 변경
    grades = np.array(['A','B', 'C'])
    print(grades[preds_arg])

    sess.close()

# 2. Dummy Feature 추가(bias를 추가)

In [6]:
def sotfmax_regression_2():
    # 맨앞에 1.,를 dummy feature 가짜 feature  bias를 없앳면서
    x = [[1., 1., 2.],  # C
         [1., 2., 1.],
         [1., 3., 5.],  # B
         [1., 5., 3.],
         [1., 7., 9.],  # A
         [1., 9., 7.]]
    # [1,0,0] A학점
    # [0,1,0] B학점
    y = [2,2,1,1,0,0]
    # y = [2,1,0]

    w = tf.Variable(tf.random_uniform([3, 3]))
    # # bias는 class의 갯수만큼 정한다.
    # b = tf.Variable(tf.random_uniform([3]))
    # print(w)

    # (6, 3) = (6, 3) @  (3, 3)
    z = tf.matmul(x, w)
    hx = tf.nn.softmax(z)

    # tensorflow 2.0d에서 정리 될것,  logit=예측  label을 결과괌
    # logit에 z를 하는것은 hx를 전달하지 않아도 됨, 자체적으로 함
    loss_i = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=z, labels=y)
    loss = tf.reduce_mean(loss_i)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    train = optimizer.minimize(loss=loss)

    # sess로 변수의 값을 알수 있다.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for i in range(100):
        sess.run(train)
        print(i, sess.run(loss))

    preds = sess.run(hx)
    preds_arg = np.argmax(preds, axis=1)  # 1: 수평, 0: 수직

    equals = (preds_arg ==y)
    print('acc: ', np.mean(equals))

    sess.close()


# 3. One-Hot 백터를 컴퓨터에 맡긴다. → sparse_softmax_cross_entropy_with_logits 사용

In [7]:
def sotfmax_regression_3():
    # 맨앞에 1.,를 dummy feature 가짜 feature  bias를 없앳면서
    x = [[1., 1., 2.],  # C
         [1., 2., 1.],
         [1., 3., 5.],  # B
         [1., 5., 3.],
         [1., 7., 9.],  # A
         [1., 9., 7.]]
    # [1,0,0] A학점
    # [0,1,0] B학점
    y = [2, 2, 1, 1, 0, 0]
    # y = [2,1,0]

    w = tf.Variable(tf.random_uniform([3, 3]))

    ph_x = tf.placeholder(tf.float32)

    # # bias는 class의 갯수만큼 정한다.
    # b = tf.Variable(tf.random_uniform([3]))
    # print(w)

    # (6, 3) = (6, 3) @  (3, 3)
    z = tf.matmul(ph_x, w)
    hx = tf.nn.softmax(z)

    # tensorflow 2.0d에서 정리 될것,  logit=예측  label을 결과괌
    # logit에 z를 하는것은 hx를 전달하지 않아도 됨, 자체적으로 함
    loss_i = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=z, labels=y)
    loss = tf.reduce_mean(loss_i)

    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    train = optimizer.minimize(loss=loss)

    # sess로 변수의 값을 알수 있다.
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for i in range(100):
        sess.run(train, feed_dict={ph_x: x})
        print(i, sess.run(loss, {ph_x: x}))

    preds = sess.run(hx, {ph_x: [[1., 4., 6.],
                                [1., 8., 3.]]})
    preds_arg = np.argmax(preds, axis=1)  # 1: 수평, 0: 수직

    # 파이썬의 list는 broadcasting기능이 없어서, numpy array로 변경
    grades = np.array(['A','B', 'C'])
    print(grades[preds_arg])
    print(preds)

    sess.close()


In [8]:
# sotfmax_regression_2()
# sotfmax_regression_2()
sotfmax_regression_3()

Instructions for updating:
Colocations handled automatically by placer.
0 1.5110563
1 0.98248667
2 0.97303134
3 0.9679238
4 0.9607339
5 0.9570642
6 0.9493826
7 0.9467001
8 0.938343
9 0.93658394
10 0.9274175
11 0.9264131
12 0.91645366
13 0.915987
14 0.9054033
15 0.9052592
16 0.89430124
17 0.89429766
18 0.88321924
19 0.88321966
20 0.872232
21 0.8721411
22 0.8613982
23 0.8611548
24 0.850759
25 0.8503259
26 0.84033984
27 0.8396969
28 0.83015376
29 0.829293
30 0.82020783
31 0.819129
32 0.8105028
33 0.8092112
34 0.801037
35 0.79954153
36 0.79180646
37 0.79011863
38 0.78280663
39 0.78093976
40 0.7740321
41 0.77200013
42 0.7654769
43 0.7632951
44 0.75713557
45 0.7548191
46 0.7490022
47 0.74656683
48 0.7410713
49 0.7385323
50 0.7333376
51 0.7307101
52 0.72579557
53 0.7230947
54 0.71844053
55 0.7156806
56 0.711267
57 0.7084623
58 0.70427114
59 0.70143443
60 0.6974478
61 0.6945916
62 0.6907926
63 0.6879286
64 0.6843012
65 0.6814401
66 0.6779693
67 0.6751203
68 0.67179227
69 0.6689639
70 0.6657657